## Analyse Promotions and Provide Tangible Insights to Sales Director

### Problem Statement
AtliQ Mart is a retail giant with over 50 supermarkets in the southern region of India. All their 50 stores ran a massive promotion during the Diwali 2023 and Sankranti 2024 (festive time in India) on their AtliQ branded products. Now the sales director wants to understand which promotions did well and which did not so that they can make informed decisions for their next promotional period.  

Sales director Bruce Haryali wanted this immediately but the analytics manager Tony is engaged on another critical project. Tony decided to give this work to Peter Pandey who is the curious data analyst of AtliQ Mart. Since these insights will be directly reported to the sales director, Tony also provided some notes to Peter to support his work.

#### Task: 

Imagine yourself as Peter Pandey and perform the following task to keep up the trust with your manager Tony Sharma. 

- Go through the metadata and analyze the datasets thoroughly. This is the most fundamental step. 
- Check “Recommended Insights.pdf” – this document contains a few recommendations from your manager Tony. 
- Design a dashboard with your metrics and analysis. The dashboard should be self-explanatory and easy to understand.
- Check “ad-hoc-requests.pdf” - this document includes important business questions posed by senior executives, requiring SQL-based report generation.
- You need to present this to the sales director - hence you need to create a convincing presentation with actionable insights.  
- You can add more research questions and answer them in your presentation that suits your recommendations.  
- Be creative with your presentation, audio/video presentation will have more weightage.   

### Recommendations
#### Store Performance Analysis:

- Which are the top 10 stores in terms of Incremental Revenue (IR) generated from the promotions?
- Which are the bottom 10 stores when it comes to Incremental Sold Units (ISU) during the promotional period?
- How does the performance of stores vary by city? Are there any common characteristics among the top performing stores that could be leveraged across other stores?

#### Promotion Type Analysis:

- What are the top 2 promotion types that resulted in the highest Incremental Revenue?
- What are the bottom 2 promotion types in terms of their impact on Incremental Sold Units?
- Is there a significant difference in the performance of discount-based promotions versus BOGOF (Buy One Get One Free) or cashback promotions?
- Which promotions strike the best balance between Incremental Sold Units and maintaining healthy margins?

#### Product and Category Analysis:

- Which product categories saw the most significant lift in sales from thepromotions?
- Are there specific products that respond exceptionally well or poorly to promotions?
- What is the correlation between product category and promotion type effectiveness?

#### Info about Dataframes
##### **Dim_Stores_DF**
- Has 50 Different Store ID's
- For each Store, we have 30 Events / Records
- If we take a Store there will be 2 campaigns which will be happening so there are 15 records for each Campaign.
- 30 Events --> Divided as 15 for each Sankranthi & Diwali,, then for Each Product_Name(Code),, there is One Campaign / Promotion helded

#### Imports

In [1]:
import pandas as pd

In [2]:
dim_campaigns_df = pd.read_csv('dim_campaigns.csv')
dim_products_df = pd.read_csv('dim_products.csv')
dim_stores_df = pd.read_csv('dim_stores.csv')
fact_events_df = pd.read_csv('fact_events.csv')

In [3]:
# Checking Shapes of the Datasets
dim_campaigns_df.shape, dim_products_df.shape, dim_stores_df.shape, fact_events_df.shape

((2, 4), (15, 3), (50, 2), (1500, 8))

In [4]:
# Getting Heads

In [5]:
dim_campaigns_df.head()

,campaign_id,campaign_name,start_date,end_date
0,CAMP_DIW_01,Diwali,12-11-2023,18-11-2023
1,CAMP_SAN_01,Sankranti,10-01-2024,16-01-2024


In [6]:
dim_products_df.sample(5)

,product_code,product_name,category
6,P09,Atliq_Body_Milk_Nourishing_Lotion (120ML),Personal Care
1,P02,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples
3,P04,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
10,P11,Atliq_Doodh_Kesar_Body_Lotion (200ML),Personal Care
7,P10,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care


In [7]:
dim_stores_df.sample(5)

,store_id,city
31,STBLR-3,Bengaluru
6,STBLR-2,Bengaluru
24,STCHE-1,Chennai
5,STBLR-8,Bengaluru
28,STBLR-9,Bengaluru


In [111]:
fact_events_df.sample(5)

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city
1329,583386,STBLR-9,Sankranthi,P08,1190,BOGOF,63,254,Bengaluru
594,078a6a,STMDU-0,Sankranthi,P09,90,25% OFF,57,47,Madurai
202,591730,STBLR-2,Sankranthi,P09,90,25% OFF,64,53,Bengaluru
717,ce3c11,STHYD-4,Diwali,P14,1020,BOGOF,52,176,Hyderabad
230,c917a3,STCHE-3,Diwali,P04,290,25% OFF,350,311,Chennai


#### Let's replace the values which are present in the Campaign_ID Column of fact_stores data frame

In [9]:
fact_events_df['campaign_id'] = fact_events_df['campaign_id'].replace(to_replace=['CAMP_DIW_01','CAMP_SAN_01'], value=['Diwali','Sankranthi'])

In [10]:
len(fact_events_df['store_id'].value_counts())

50

#### Merging dim_stores_df && fact_events_df

In [11]:
fact_events_df = pd.merge(fact_events_df,dim_stores_df,on='store_id',how='left')

In [12]:
#### Let's replace Store ID values with their exact names in fact_events_df

# Not Correct

# Creating a dictionary which contains Store_id as key and City as value
# store_id_city_mapping_dict = dict(zip(dim_stores_df['store_id'], dim_stores_df['city']))

In [13]:
# fact_events_df['store_id'] = fact_events_df['store_id'].map(store_id_city_mapping_dict)

#### Merging dim_products_df & fact_events_df table such that in the final data frame will have product_name and product_category as per product_code

In [14]:
df = pd.merge(fact_events_df, dim_products_df, on='product_code', how='left')

In [15]:
df.sample(8)

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category
1443,c9fa13,STHYD-4,Sankranthi,P04,370,BOGOF,403,1567,Hyderabad,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples
782,5.00E+300,STVJD-1,Diwali,P14,1020,BOGOF,24,80,Vijayawada,Atliq_waterproof_Immersion_Rod,Home Appliances
551,fc8d10,STMLR-0,Sankranthi,P08,1190,BOGOF,27,70,Mangalore,Atliq_Double_Bedsheet_set,Home Care
899,f9d2c6,STBLR-9,Sankranthi,P14,1020,BOGOF,111,429,Bengaluru,Atliq_waterproof_Immersion_Rod,Home Appliances
896,dec49d,STMDU-2,Sankranthi,P05,55,25% OFF,21,17,Madurai,Atliq_Scrub_Sponge_For_Dishwash,Home Care
1300,5f8870,STMYS-1,Diwali,P13,350,BOGOF,70,270,Mysuru,Atliq_High_Glo_15W_LED_Bulb,Home Appliances
881,0ae76a,STMLR-1,Sankranthi,P11,190,50% OFF,21,30,Mangalore,Atliq_Doodh_Kesar_Body_Lotion (200ML),Personal Care
512,b76f57,STBLR-8,Diwali,P13,350,BOGOF,66,220,Bengaluru,Atliq_High_Glo_15W_LED_Bulb,Home Appliances


In [16]:
# Downloading Df which has Proper Mergings

df.to_csv('df.csv')

#### Value Counts of Campaign

In [17]:
fact_events_df['campaign_id'].value_counts()

Sankranthi    750
Diwali        750
Name: campaign_id, dtype: int64

#### Value Counts of Store_ID

In [18]:
fact_events_df['city'].value_counts()

Bengaluru        300
Chennai          240
Hyderabad        210
Coimbatore       150
Visakhapatnam    150
Madurai          120
Mysuru           120
Mangalore         90
Vijayawada        60
Trivandrum        60
Name: city, dtype: int64

#### Value Counts of Promo_Type

In [19]:
fact_events_df['promo_type'].value_counts()

BOGOF           500
25% OFF         400
50% OFF         300
33% OFF         200
500 Cashback    100
Name: promo_type, dtype: int64

#### Number of Products under each Category

In [20]:
dim_products_df.groupby('category').count()['product_code']

category
Combo1               1
Grocery & Staples    4
Home Appliances      2
Home Care            4
Personal Care        4
Name: product_code, dtype: int64

In [21]:
# dim_products_df.shape # (15, 3)

In [22]:
df.sample(5)

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category
131,00a25a,STCHE-3,Sankranthi,P12,62,50% OFF,64,89,Chennai,Atliq_Lime_Cool_Bathing_Bar (125GM),Personal Care
332,27bc6e,STCBE-2,Diwali,P14,1020,BOGOF,42,160,Coimbatore,Atliq_waterproof_Immersion_Rod,Home Appliances
277,5.88E+11,STHYD-3,Diwali,P12,62,50% OFF,112,178,Hyderabad,Atliq_Lime_Cool_Bathing_Bar (125GM),Personal Care
507,98cc83,STVSK-0,Diwali,P10,65,50% OFF,84,127,Visakhapatnam,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care
1386,da3d1d,STVSK-2,Diwali,P06,415,25% OFF,78,69,Visakhapatnam,Atliq_Fusion_Container_Set_of_3,Home Care


#### **Calculating Effective Price during Campaign / Promotion**
- Promo_Type is **"BOGOF"** ,We can consider the Effective Price as the Original Base Price divided by 2 since Customers are essentially getting 2 Items for the Price of 1.
- If the Promo_Type involves a **Percentage Discount (e.g., "25% OFF")**, We can calculate the Effective price by Subtracting the Discount amount from the Base Price.
- For 500 CashBack,, We can subtract best price with 500.

In [23]:
# Getting exact Percentage value from the promo_type String
# float(''.join(filter(str.isdigit, '25% OFF'))) # 25.0

In [24]:
# Define a function to calculate effective price based on promo_type
def calculate_effective_price(row):
    if row['promo_type'] == 'BOGOF':
        return row['base_price'] / 2
    elif 'OFF' in row['promo_type']:
        # Extract the percentage discount from promo_type
        discount_percentage = float(''.join(filter(str.isdigit, row['promo_type'])))
        return row['base_price'] * (1 - discount_percentage / 100)
    elif 'Cashback' in row['promo_type']:
        # Extract the cashback amount from promo_type
        cashback_amount = float(''.join(filter(str.isdigit, row['promo_type'])))
        return row['base_price'] - cashback_amount
    else:
        return row['base_price']

In [25]:
# Apply the function to create a new column for effective price
df['effective_price'] = df.apply(calculate_effective_price, axis=1)

In [26]:
df[['base_price','promo_type','effective_price']].sample(8)

,base_price,promo_type,effective_price
430,415,25% OFF,311.25
1107,62,50% OFF,31.00
626,3000,500 Cashback,2500.00
429,1190,BOGOF,595.00
1207,62,50% OFF,31.00
1357,350,BOGOF,175.00
802,65,50% OFF,32.50
791,860,33% OFF,576.20


#### **Price Sensitivity Analysis Before and After Promotion**

##### **Calculate Percentage Change in Quantity Sold**

In [27]:
df['quantity_change_percentage'] = (
    (df['quantity_sold(after_promo)'] - df['quantity_sold(before_promo)']) / df['quantity_sold(before_promo)']
) * 100

In [28]:
# df[['quantity_sold(before_promo)','quantity_sold(after_promo)','quantity_change_percentage']].sample(5)

In [29]:
df[['base_price','effective_price']].sample(5)

,base_price,effective_price
1149,300,150.00
1366,3000,2500.00
1134,55,41.25
17,172,115.24
943,290,217.50


### **Top 10 Stores in terms of Incremental Revenue (IR) generated from the promotions**
##### **As after Giving Discont,, Generally Effective Price will decrease.**

In [30]:
# May be Wrong Consideration

# Assuming you have a DataFrame with 'base_price', 'effective_price', and 'quantity_sold_after_promo' columns

# df['incremental_revenue'] = (df['effective_price'] - df['base_price']) * df['quantity_sold(after_promo)']

In [31]:
# Let's check the value counts of store IDs
# df['store_id'].value_counts() # For each Store_ID,, Count is 30
# Total Number of Unique store IDs is 50

In [32]:
# As after Giving Discont,, Generally Effective Price will decrease.
# df.groupby('store_id')['incremental_revenue'].sum().reset_index()

In [33]:
df['revenue_before_promo'] = df['quantity_sold(before_promo)'] * df['base_price']
df['revenue_after_promo'] = df['quantity_sold(after_promo)'] * df['effective_price']

In [34]:
# Calculate Incremental Revenue
df['incremental_revenue'] = df['revenue_after_promo'] - df['revenue_before_promo']

In [35]:
# Group the data by store and sum the Incremental Revenue

all_stores_incremental_revenue = df.groupby('store_id')['incremental_revenue'].sum().reset_index()

In [36]:
# Sort the stores by Incremental Revenue in descending order

all_stores_incremental_revenue.sort_values(by='incremental_revenue', ascending=False)[:10]

,store_id,incremental_revenue
38,STMYS-1,3634103.54
19,STCHE-4,3536630.03
0,STBLR-0,3506983.65
7,STBLR-7,3397407.33
40,STMYS-3,3362866.98
22,STCHE-7,3338660.72
6,STBLR-6,3292205.29
18,STCHE-3,3155340.33
30,STMDU-0,2829717.01
3,STBLR-3,2780073.40


In [37]:
# Downloading All_Stores_Incremental Revenue Data

all_stores_incremental_revenue.sort_values(by='incremental_revenue', ascending=False).to_csv('all_stores_incremental_revenue.csv')

In [38]:
# all_stores_incremental_revenue.sort_values(by='incremental_revenue', ascending=False)

#### **Bottom 10 stores when it comes to Incremental Sold Units (ISU) during the Promotional Period**

In [39]:
# Calculate ISU for each record

df['ISU'] = df['quantity_sold(after_promo)'] - df['quantity_sold(before_promo)']

In [40]:
# df.groupby('store_id')['ISU'].sum().reset_index()

In [41]:
# Sum ISU for each store
total_isu_by_store = df.groupby('store_id')['ISU'].sum().reset_index()

In [42]:
# Rank stores based on total ISU in ascending order
total_isu_by_store.sort_values(by='ISU', ascending=True)[:3]

,store_id,ISU
34,STMLR-0,1952
48,STVSK-3,2209
49,STVSK-4,2469


In [43]:
total_isu_by_store.sort_values(by='ISU', ascending=True).to_csv('ISU_store_wise.csv')

#### **Performance of Stores based on City**

**Let's See which Revenues of Each City,, Before Promo & After Promo && See the Change**

##### **Getting Number of Stores in a City**

In [44]:
dim_stores_df['city'].value_counts()

Bengaluru        10
Chennai           8
Hyderabad         7
Visakhapatnam     5
Coimbatore        5
Madurai           4
Mysuru            4
Mangalore         3
Trivandrum        2
Vijayawada        2
Name: city, dtype: int64

#### Grouping Only by City

In [45]:
# AVERAGE Incremental Revenue
df.groupby('city')['incremental_revenue'].mean().reset_index().sort_values(by='incremental_revenue', ascending=False)

,city,incremental_revenue
0,Bengaluru,89143.865967
1,Chennai,88187.267333
6,Mysuru,83923.919000
4,Madurai,74740.633417
3,Hyderabad,72555.107095
2,Coimbatore,62074.719933
9,Visakhapatnam,48121.480267
8,Vijayawada,47588.302000
7,Trivandrum,39188.091833
5,Mangalore,37365.657778


In [46]:
# TOTAL Incremental Revenue
df.groupby('city')['incremental_revenue'].sum().reset_index().sort_values(by='incremental_revenue', ascending=False)

,city,incremental_revenue
0,Bengaluru,26743159.79
1,Chennai,21164944.16
3,Hyderabad,15236572.49
6,Mysuru,10070870.28
2,Coimbatore,9311207.99
4,Madurai,8968876.01
9,Visakhapatnam,7218222.04
5,Mangalore,3362909.20
8,Vijayawada,2855298.12
7,Trivandrum,2351285.51


In [47]:
df.groupby('city')['incremental_revenue'].mean().reset_index().sort_values(by='incremental_revenue', ascending=False).to_csv('avg_IR_city_wise.csv')

#### Grouping by City & Store

In [48]:
# Sorting the Stores Based on IR
df.groupby(['city', 'store_id'])['incremental_revenue'].sum().groupby('city', group_keys=False).apply(lambda x: x.sort_values(ascending=False))

city           store_id
Bengaluru      STBLR-0     3506983.65
               STBLR-7     3397407.33
               STBLR-6     3292205.29
               STBLR-3     2780073.40
               STBLR-8     2684009.68
               STBLR-4     2626485.53
               STBLR-5     2494656.81
               STBLR-9     2289928.59
               STBLR-2     1984680.78
               STBLR-1     1686728.73
Chennai        STCHE-4     3536630.03
               STCHE-7     3338660.72
               STCHE-3     3155340.33
               STCHE-6     2741370.71
               STCHE-5     2471319.71
               STCHE-0     2346568.20
               STCHE-2     2196610.82
               STCHE-1     1378443.64
Coimbatore     STCBE-2     2205103.90
               STCBE-0     2106919.72
               STCBE-1     1995644.01
               STCBE-3     1723738.41
               STCBE-4     1279801.95
Hyderabad      STHYD-0     2670264.12
               STHYD-2     2509073.99
               STHYD-6    

In [49]:
# Comparing the Sum & Avg of IR for Stores in a City
df.groupby(['city', 'store_id'])['incremental_revenue'].sum().loc['Visakhapatnam'].sort_values(ascending=False)

store_id
STVSK-1    1766942.14
STVSK-0    1725451.23
STVSK-2    1491984.09
STVSK-4    1178448.11
STVSK-3    1055396.47
Name: incremental_revenue, dtype: float64

In [50]:
# Getting Percentage of Total For Each Store in IR for Visakhapatnam City ( For comparing with Tableau )

data = df.groupby(['city', 'store_id'])['incremental_revenue'].sum().loc['Visakhapatnam'].sort_values(ascending=False)

# Calculate the percentage of total incremental revenue
total_revenue = data.sum()
percentage_of_total = (data / total_revenue) * 100

# Create a DataFrame to display the results
result_df = pd.DataFrame({
    'Incremental Revenue': data,
    'Percentage of Total': percentage_of_total
})

# Sort the DataFrame by Incremental Revenue in descending order
result_df = result_df.sort_values(by='Incremental Revenue', ascending=False)

# Display the result
print(result_df)


          Incremental Revenue  Percentage of Total
store_id                                          
STVSK-1            1766942.14            24.478911
STVSK-0            1725451.23            23.904103
STVSK-2            1491984.09            20.669690
STVSK-4            1178448.11            16.326016
STVSK-3            1055396.47            14.621280


In [51]:
# Comparing the Sum & Avg of IR for Stores in a City
df.groupby(['city', 'store_id'])['incremental_revenue'].mean().loc['Visakhapatnam'].sort_values(ascending=False)

store_id
STVSK-1    58898.071333
STVSK-0    57515.041000
STVSK-2    49732.803000
STVSK-4    39281.603667
STVSK-3    35179.882333
Name: incremental_revenue, dtype: float64

In [52]:
# Saving city_store_wise_IR.csv File

df.groupby(['city', 'store_id'])['incremental_revenue'].sum().groupby('city', group_keys=False).apply(lambda x: x.sort_values(ascending=False)).reset_index().to_csv('city_store_wise_IR.csv')

#### What analysis can be done based on city
- Analyze the quantity change percentage for each city to understand how effective the promotions were.
- Evaluate the distribution of promo types across cities. This will help you understand which types of promotions are more popular in specific locations.
- Explore the distribution of product categories in each city to identify the most popular categories in different regions
- Visualize the distribution of quantity sold (before and after promo) for each city to identify outliers or patterns.
- Analyze the performance of campaigns in each city to identify which campaigns were more successful in different regions

In [53]:
df.sample(5)

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category,effective_price,quantity_change_percentage,revenue_before_promo,revenue_after_promo,incremental_revenue,ISU
478,fe2fe7,STCHE-4,Diwali,P05,55,25% OFF,127,115,Chennai,Atliq_Scrub_Sponge_For_Dishwash,Home Care,41.25,-9.448819,6985,4743.75,-2241.25,-12
294,f1b7e9,STCHE-3,Diwali,P14,1020,BOGOF,52,211,Chennai,Atliq_waterproof_Immersion_Rod,Home Appliances,510.00,305.769231,53040,107610.00,54570.00,159
1270,65f85a,STBLR-7,Sankranthi,P12,62,50% OFF,58,92,Bengaluru,Atliq_Lime_Cool_Bathing_Bar (125GM),Personal Care,31.00,58.620690,3596,2852.00,-744.00,34
1320,c0929c,STHYD-0,Sankranthi,P05,55,25% OFF,18,14,Hyderabad,Atliq_Scrub_Sponge_For_Dishwash,Home Care,41.25,-22.222222,990,577.50,-412.50,-4
591,b1a501,STBLR-0,Diwali,P11,190,50% OFF,80,108,Bengaluru,Atliq_Doodh_Kesar_Body_Lotion (200ML),Personal Care,95.00,35.000000,15200,10260.00,-4940.00,28


### City wise Ordering Stores based on Quantity Change Percentage

In [54]:
df.groupby(['city'])['quantity_change_percentage'].mean()

city
Bengaluru        112.659671
Chennai          110.415157
Coimbatore       107.943333
Hyderabad        100.984063
Madurai          111.065352
Mangalore         95.846612
Mysuru           100.661537
Trivandrum       110.892932
Vijayawada       110.032113
Visakhapatnam     95.890340
Name: quantity_change_percentage, dtype: float64

In [55]:
df.groupby(['city','store_id'])['quantity_change_percentage'].mean()

city           store_id
Bengaluru      STBLR-0     125.676440
               STBLR-1      78.747383
               STBLR-2     109.969793
               STBLR-3     107.039788
               STBLR-4     110.606942
               STBLR-5     109.967688
               STBLR-6     130.611816
               STBLR-7     133.736571
               STBLR-8     107.148923
               STBLR-9     113.091372
Chennai        STCHE-0     110.225896
               STCHE-1      67.965892
               STCHE-2     105.534202
               STCHE-3     127.632854
               STCHE-4     128.799131
               STCHE-5     104.396491
               STCHE-6     106.586013
               STCHE-7     132.180779
Coimbatore     STCBE-0     117.668790
               STCBE-1     100.656303
               STCBE-2     131.650506
               STCBE-3     114.177733
               STCBE-4      75.563331
Hyderabad      STHYD-0     115.897344
               STHYD-1      69.190549
               STHYD-2    

In [56]:
# df[(df['city'] == 'Bengaluru') & (df['store_id'] == 'STBLR-1')] # 30 Records

In [57]:
df.to_csv('ISU_included.csv')

In [58]:
df[(df['city'] == 'Bengaluru') & (df['store_id'] == 'STBLR-1')]['ISU'].sum() # 30 Records

3770

In [59]:
df[(df['city'] == 'Bengaluru') & (df['store_id'] == 'STBLR-1')]['ISU'].sum()

3770

#### **Distribution of Promo Types in Each City**

In [60]:
# Checking Value_counts for Each Promo Type in Bengaluru

df[(df['city'] == 'Bengaluru')]['promo_type'].value_counts()

BOGOF           100
25% OFF          80
50% OFF          60
33% OFF          40
500 Cashback     20
Name: promo_type, dtype: int64

In [61]:
df[(df['city'] == 'Mangalore')]['promo_type'].value_counts(normalize=False)

BOGOF           30
25% OFF         24
50% OFF         18
33% OFF         12
500 Cashback     6
Name: promo_type, dtype: int64

In [62]:
# Checking Value_counts for Each Promo Type in Hyderabad

df[(df['city'] == 'Hyderabad')]['promo_type'].value_counts(normalize=True)

BOGOF           0.333333
25% OFF         0.266667
50% OFF         0.200000
33% OFF         0.133333
500 Cashback    0.066667
Name: promo_type, dtype: float64

#### **Getting Promo_Type through which more Incremental Revenue is Getting generated**

In [63]:
df[(df['city'] == 'Bengaluru')].groupby('promo_type')['incremental_revenue'].sum()

promo_type
25% OFF          -752193.25
33% OFF          -143177.96
50% OFF          -157429.00
500 Cashback    22348000.00
BOGOF            5447960.00
Name: incremental_revenue, dtype: float64

In [64]:
# TOTAL Incremental Revenue based on a Promo Type of '33% OFF'
df[(df['city'] == 'Bengaluru') & (df['promo_type'] == '33% OFF')]['incremental_revenue'].sum()

-143177.960000001

In [65]:
# AVERAGE Incremental Revenue on a Promo Type of '33% OFF'
df[(df['city'] == 'Bengaluru') & (df['promo_type'] == '33% OFF')]['incremental_revenue'].mean()

-3579.449000000025

#### **Bottom 2 Promotion Types in terms of their Impact on Incremental Sold Units**

In [66]:
df[(df['city'] == 'Bengaluru') & (df['promo_type'] == '25% OFF')][['quantity_sold(before_promo)','quantity_sold(after_promo)','ISU','quantity_change_percentage']].sample(2)

,quantity_sold(before_promo),quantity_sold(after_promo),ISU,quantity_change_percentage
549,72,58,-14,-19.444444
436,276,242,-34,-12.318841


In [67]:
df.columns

Index(['event_id', 'store_id', 'campaign_id', 'product_code', 'base_price',
       'promo_type', 'quantity_sold(before_promo)',
       'quantity_sold(after_promo)', 'city', 'product_name', 'category',
       'effective_price', 'quantity_change_percentage', 'revenue_before_promo',
       'revenue_after_promo', 'incremental_revenue', 'ISU'],
      dtype='object')

In [68]:
df.groupby(['city','promo_type'])['ISU'].sum()

city           promo_type  
Bengaluru      25% OFF         -1350
               33% OFF          6934
               50% OFF          1814
               500 Cashback     9991
               BOGOF           38581
Chennai        25% OFF          -970
               33% OFF          4342
               50% OFF          1050
               500 Cashback     8001
               BOGOF           31345
Coimbatore     25% OFF          -513
               33% OFF          2671
               50% OFF           671
               500 Cashback     3453
               BOGOF           14468
Hyderabad      25% OFF          -940
               33% OFF          4700
               50% OFF          1291
               500 Cashback     5845
               BOGOF           24140
Madurai        25% OFF          -382
               33% OFF          1480
               50% OFF           311
               500 Cashback     3485
               BOGOF           11817
Mangalore      25% OFF          -221
          

#### **Most Popular Product Categories in City Wise(Happens in Tableau)**
* There will be Some Product Categories which had a Better Sold Units Both Before and After the Promo..
* There  will be Some Categories whose Percentage of Buying (quantity) had increased a lot after Promo.

Let's Capture them

In [69]:
df[df['city'] == 'Chennai'].groupby('category')['quantity_sold(before_promo)'].sum().sort_values(ascending=False)

category
Grocery & Staples    23872
Personal Care         4977
Combo1                4080
Home Care             3759
Home Appliances       2817
Name: quantity_sold(before_promo), dtype: int64

In [70]:
df[df['city'] == 'Chennai'].groupby('category')['quantity_sold(after_promo)'].sum().sort_values(ascending=False)

category
Grocery & Staples    46639
Combo1               12081
Home Appliances      10686
Home Care             7974
Personal Care         5893
Name: quantity_sold(after_promo), dtype: int64

In [71]:
df[df['city'] == 'Chennai'].groupby('category')['quantity_change_percentage'].sum().sort_values(ascending=False)

category
Home Appliances      8774.019276
Home Care            8265.983486
Grocery & Staples    5667.488198
Combo1               2823.906120
Personal Care         968.240665
Name: quantity_change_percentage, dtype: float64

In [72]:
# df.groupby(['city','category'])[['quantity_sold(before_promo)','quantity_sold(after_promo)']].sum()

In [73]:
# Group by 'city' and 'category' and sum the quantities
result = df.groupby(['city', 'category'])[['quantity_sold(before_promo)', 'quantity_sold(after_promo)']].sum()

# Add a new column 'quantity_difference' to store the difference between before and after promo
result['quantity_difference'] = result['quantity_sold(after_promo)'] - result['quantity_sold(before_promo)']

# Display the result
# result

In [74]:
# result.groupby(['city']).apply(lambda x: x.sort_values(by='quantity_difference', ascending=False)).reset_index(level=0, drop=True)

In [75]:
result.groupby(['city']).apply(lambda x: x.sort_values(by='quantity_difference', ascending=False)).reset_index(level=0, drop=True).to_csv('category_city_wise_Sold_units_before_after.csv')

#### **City wise Which Campaign became more Successful**

In [76]:
result = df.groupby(['city', 'campaign_id'])['incremental_revenue'].sum()

# Sort the DataFrame within each group based on 'incremental_revenue'
result_sorted = result.groupby(['city']).apply(lambda x: x.sort_values(ascending=False))

# Reset the index to remove the repeating 'city' index column
result_sorted = result_sorted.reset_index(level=0, drop=True)
result_sorted

/var/folders/yz/rd_p5r694wz4_q5mc3q3wns40000gn/T/ipykernel_1358/2528033958.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result_sorted = result.groupby(['city']).apply(lambda x: x.sort_values(ascending=False))


campaign_id
Diwali        19168351.09
Sankranthi     7574808.70
Diwali        15141007.21
Sankranthi     6023936.95
Diwali         6637630.49
Sankranthi     2673577.50
Diwali        11155118.23
Sankranthi     4081454.26
Diwali         6499925.29
Sankranthi     2468950.72
Diwali         2568787.56
Sankranthi      794121.64
Diwali         7562084.19
Sankranthi     2508786.09
Diwali         1617331.85
Sankranthi      733953.66
Diwali         2091510.34
Sankranthi      763787.78
Diwali         5273216.73
Sankranthi     1945005.31
Name: incremental_revenue, dtype: float64

#### **OverAll IR based on 4 Promo Types**

In [77]:
df.groupby('promo_type')['incremental_revenue'].sum()

promo_type
25% OFF         -3174514.75
33% OFF         -1563356.16
50% OFF          -726663.50
500 Cashback    91053000.00
BOGOF           21694880.00
Name: incremental_revenue, dtype: float64

In [78]:
df.columns

Index(['event_id', 'store_id', 'campaign_id', 'product_code', 'base_price',
       'promo_type', 'quantity_sold(before_promo)',
       'quantity_sold(after_promo)', 'city', 'product_name', 'category',
       'effective_price', 'quantity_change_percentage', 'revenue_before_promo',
       'revenue_after_promo', 'incremental_revenue', 'ISU'],
      dtype='object')

#### **Finding which Promo_Type is better for which category**

In [79]:
df[['category','promo_type','incremental_revenue']]

,category,promo_type,incremental_revenue
0,Personal Care,50% OFF,-1520.00
1,Grocery & Staples,25% OFF,-23634.00
2,Home Care,BOGOF,6150.00
3,Combo1,500 Cashback,1513000.00
4,Home Care,25% OFF,-2103.75
...,...,...,...
1495,Personal Care,50% OFF,-1178.00
1496,Personal Care,25% OFF,-425.00
1497,Personal Care,50% OFF,-1271.00
1498,Home Appliances,BOGOF,23800.00


In [80]:
df[['category','promo_type','quantity_change_percentage']].groupby(['category','promo_type'])['quantity_change_percentage'].sum()

category           promo_type  
Combo1             500 Cashback    16217.093940
Grocery & Staples  25% OFF         -1205.657354
                   33% OFF          8653.296910
                   BOGOF           27445.765175
Home Appliances    BOGOF           51613.068632
Home Care          25% OFF         -3132.853532
                   BOGOF           51552.618998
Personal Care      25% OFF         -1821.932535
                   50% OFF         10046.767510
Name: quantity_change_percentage, dtype: float64

In [81]:
df[['category','promo_type','incremental_revenue']].groupby(['category','promo_type'])['incremental_revenue'].sum()

category           promo_type  
Combo1             500 Cashback    91053000.00
Grocery & Staples  25% OFF         -2175406.00
                   33% OFF         -1563356.16
                   BOGOF            8833350.00
Home Appliances    BOGOF            7833640.00
Home Care          25% OFF          -880273.75
                   BOGOF            5027890.00
Personal Care      25% OFF          -118835.00
                   50% OFF          -726663.50
Name: incremental_revenue, dtype: float64

#### **Trying to find Demanding Product Categories in Sankranthi & Diwali**

In [82]:
df.sample(10)

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category,effective_price,quantity_change_percentage,revenue_before_promo,revenue_after_promo,incremental_revenue,ISU
1460,c25b15,STHYD-6,Diwali,P07,300,BOGOF,63,218,Hyderabad,Atliq_Curtains,Home Care,150.00,246.031746,18900,32700.00,13800.00,155
699,6a1564,STMDU-3,Diwali,P15,3000,500 Cashback,334,1022,Madurai,Atliq_Home_Essential_8_Product_Combo,Combo1,2500.00,205.988024,1002000,2555000.00,1553000.00,688
858,4cf56a,STBLR-9,Diwali,P04,290,25% OFF,311,304,Bengaluru,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,217.50,-2.250804,90190,66120.00,-24070.00,-7
341,dead78,STVJD-0,Diwali,P12,62,50% OFF,68,90,Vijayawada,Atliq_Lime_Cool_Bathing_Bar (125GM),Personal Care,31.00,32.352941,4216,2790.00,-1426.00,22
654,bae5cb,STVSK-3,Sankranthi,P11,190,50% OFF,34,39,Visakhapatnam,Atliq_Doodh_Kesar_Body_Lotion (200ML),Personal Care,95.00,14.705882,6460,3705.00,-2755.00,5
1132,f78e86,STCHE-3,Diwali,P06,415,25% OFF,77,67,Chennai,Atliq_Fusion_Container_Set_of_3,Home Care,311.25,-12.987013,31955,20853.75,-11101.25,-10
823,93dff2,STBLR-3,Diwali,P03,156,25% OFF,390,308,Bengaluru,Atliq_Suflower_Oil (1L),Grocery & Staples,117.00,-21.025641,60840,36036.00,-24804.00,-82
1166,24054f,STBLR-4,Diwali,P09,110,50% OFF,89,116,Bengaluru,Atliq_Body_Milk_Nourishing_Lotion (120ML),Personal Care,55.00,30.337079,9790,6380.00,-3410.00,27
140,345b49,STMDU-2,Sankranthi,P14,1020,BOGOF,76,300,Madurai,Atliq_waterproof_Immersion_Rod,Home Appliances,510.00,294.736842,77520,153000.00,75480.00,224
1441,d04921,STMLR-1,Diwali,P10,65,50% OFF,50,68,Mangalore,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care,32.50,36.000000,3250,2210.00,-1040.00,18


In [83]:
# Create a new DataFrame for Sankranthi data
sankranthi_df = df[df['campaign_id'] == 'Sankranthi']

# Create a new DataFrame for Diwali data
diwali_df = df[df['campaign_id'] == 'Diwali']

# Group by product category and sum the quantity sold for Sankranthi
sankranthi_category_demand = sankranthi_df.groupby('category')['quantity_sold(after_promo)'].sum().reset_index()

# Group by product category and sum the quantity sold for Diwali
diwali_category_demand = diwali_df.groupby('category')['quantity_sold(after_promo)'].sum().reset_index()

# Merge the two DataFrames on the 'category' column
merged_demand = pd.merge(sankranthi_category_demand, diwali_category_demand, on='category', suffixes=('_sankranthi', '_diwali'))

In [84]:
merged_demand

,category,quantity_sold(after_promo)_sankranthi,quantity_sold(after_promo)_diwali
0,Combo1,12411,50769
1,Grocery & Staples,177724,68620
2,Home Appliances,35610,18003
3,Home Care,16894,23938
4,Personal Care,9430,22074


In [85]:
merged_demand.to_csv('category_wise_quantity_change_Sankrathi_Diwali.csv')

#### **Promo Type & Campaign & ISU**

In [86]:
# ISU Sum 
df.groupby(['campaign_id','promo_type'])['ISU'].sum()

campaign_id  promo_type  
Diwali       25% OFF          -4526
             33% OFF          13965
             50% OFF           5231
             500 Cashback     33978
             BOGOF            24437
Sankranthi   25% OFF          -1191
             33% OFF          13290
             50% OFF           1700
             500 Cashback      6903
             BOGOF           132636
Name: ISU, dtype: int64

In [87]:
df_grouped = df.groupby(['campaign_id', 'promo_type']).size().unstack(fill_value=0)

In [88]:
# Value Counts of Promo_Type based  on Campaogn ID
df_grouped

promo_type,25% OFF,33% OFF,50% OFF,500 Cashback,BOGOF
campaign_id,,,,,
Diwali,200,100,200,50,200
Sankranthi,200,100,100,50,300


In [89]:
df[(df['campaign_id']=='Diwali') & (df['promo_type']=='25% OFF') ]['ISU'].sum()

-4526

In [90]:
df[(df['campaign_id']=='Sankranthi') & (df['promo_type']=='25% OFF') ]['ISU'].sum()

-1191

#### **Best Product Items under each Category**

In [91]:
df.columns

Index(['event_id', 'store_id', 'campaign_id', 'product_code', 'base_price',
       'promo_type', 'quantity_sold(before_promo)',
       'quantity_sold(after_promo)', 'city', 'product_name', 'category',
       'effective_price', 'quantity_change_percentage', 'revenue_before_promo',
       'revenue_after_promo', 'incremental_revenue', 'ISU'],
      dtype='object')

In [92]:
df[ (df['city']=='Hyderabad') & (df['campaign_id']=='Sankranthi') ]

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category,effective_price,quantity_change_percentage,revenue_before_promo,revenue_after_promo,incremental_revenue,ISU
11,8f25a6,STHYD-6,Sankranthi,P15,3000,500 Cashback,126,302,Hyderabad,Atliq_Home_Essential_8_Product_Combo,Combo1,2500.0,139.682540,378000,755000.0,377000.0,176
19,7ef92f,STHYD-5,Sankranthi,P07,300,BOGOF,55,213,Hyderabad,Atliq_Curtains,Home Care,150.0,287.272727,16500,31950.0,15450.0,158
53,a85a9a,STHYD-2,Sankranthi,P10,50,25% OFF,42,31,Hyderabad,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care,37.5,-26.190476,2100,1162.5,-937.5,-11
69,57a7bc,STHYD-5,Sankranthi,P14,1020,BOGOF,100,391,Hyderabad,Atliq_waterproof_Immersion_Rod,Home Appliances,510.0,291.000000,102000,199410.0,97410.0,291
72,1c1cb7,STHYD-5,Sankranthi,P10,50,25% OFF,39,35,Hyderabad,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care,37.5,-10.256410,1950,1312.5,-637.5,-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,c9fa13,STHYD-4,Sankranthi,P04,370,BOGOF,403,1567,Hyderabad,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,185.0,288.833747,149110,289895.0,140785.0,1164
1454,46d796,STHYD-6,Sankranthi,P07,300,BOGOF,51,205,Hyderabad,Atliq_Curtains,Home Care,150.0,301.960784,15300,30750.0,15450.0,154
1461,4f560f,STHYD-0,Sankranthi,P02,860,33% OFF,514,791,Hyderabad,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,576.2,53.891051,442040,455774.2,13734.2,277
1479,136c14,STHYD-1,Sankranthi,P03,200,BOGOF,328,833,Hyderabad,Atliq_Suflower_Oil (1L),Grocery & Staples,100.0,153.963415,65600,83300.0,17700.0,505


In [93]:
df[ (df['city']=='Hyderabad') & (df['campaign_id']=='Sankranthi') ].groupby('category')

In [94]:
df.sample(5)

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category,effective_price,quantity_change_percentage,revenue_before_promo,revenue_after_promo,incremental_revenue,ISU
485,6321fa,STCHE-3,Sankranthi,P10,50,25% OFF,31,26,Chennai,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care,37.50,-16.129032,1550,975.00,-575.00,-5
1031,c8e17c,STMLR-1,Diwali,P02,860,33% OFF,166,252,Mangalore,Atliq_Sonamasuri_Rice (10KG),Grocery & Staples,576.20,51.807229,142760,145202.40,2442.40,86
1243,5bfb5e,STCBE-4,Diwali,P10,65,50% OFF,57,72,Coimbatore,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care,32.50,26.315789,3705,2340.00,-1365.00,15
631,5760fd,STCBE-1,Sankranthi,P13,350,BOGOF,87,346,Coimbatore,Atliq_High_Glo_15W_LED_Bulb,Home Appliances,175.00,297.701149,30450,60550.00,30100.00,259
1469,d953a5,STBLR-1,Sankranthi,P01,172,33% OFF,297,418,Bengaluru,Atliq_Masoor_Dal (1KG),Grocery & Staples,115.24,40.740741,51084,48170.32,-2913.68,121


In [95]:
df.groupby(['city', 'campaign_id', 'category', 'product_name']).agg({
    'quantity_change_percentage': 'mean',
    'incremental_revenue': 'sum'
}).reset_index().groupby(['city', 'campaign_id', 'category'])['incremental_revenue'].rank(ascending=False, method='max')

0      1.0
1      4.0
2      2.0
3      1.0
4      3.0
      ... 
295    3.0
296    3.0
297    1.0
298    4.0
299    2.0
Name: incremental_revenue, Length: 300, dtype: float64

In [96]:
import pandas as pd

# Assuming df is your DataFrame

# Convert 'effective_price' and other relevant columns to numeric
# df['effective_price'] = pd.to_numeric(df['effective_price'])
# df['quantity_change_percentage'] = pd.to_numeric(df['quantity_change_percentage'])
# df['incremental_revenue'] = pd.to_numeric(df['incremental_revenue'])

# Group by city, campaign, and category and sum the sales metrics
grouped_df = df.groupby(['city', 'campaign_id', 'category', 'product_name']).agg({
    'quantity_change_percentage': 'mean',
    'incremental_revenue': 'sum'
}).reset_index()

# Rank products within each category based on incremental revenue
grouped_df['rank'] = grouped_df.groupby(['city', 'campaign_id', 'category'])['incremental_revenue'].rank(ascending=False, method='max')

# Filter for the top 10 products in each category
top_products = grouped_df[grouped_df['rank'] <= 10]

# Display the result
top_products

,city,campaign_id,category,product_name,quantity_change_percentage,incremental_revenue,rank
0,Bengaluru,Diwali,Combo1,Atliq_Home_Essential_8_Product_Combo,208.270351,18710000.00,1.0
1,Bengaluru,Diwali,Grocery & Staples,Atliq_Farm_Chakki_Atta (1KG),-12.423088,-324292.50,4.0
2,Bengaluru,Diwali,Grocery & Staples,Atliq_Masoor_Dal (1KG),51.184171,6358.84,2.0
3,Bengaluru,Diwali,Grocery & Staples,Atliq_Sonamasuri_Rice (10KG),52.738691,69488.00,1.0
4,Bengaluru,Diwali,Grocery & Staples,Atliq_Suflower_Oil (1L),-11.227077,-191997.00,3.0
...,...,...,...,...,...,...,...
295,Visakhapatnam,Sankranthi,Home Care,Atliq_Scrub_Sponge_For_Dishwash,-23.473684,-2090.00,3.0
296,Visakhapatnam,Sankranthi,Personal Care,Atliq_Body_Milk_Nourishing_Lotion (120ML),-22.450945,-7425.00,3.0
297,Visakhapatnam,Sankranthi,Personal Care,Atliq_Cream_Beauty_Bathing_Soap (125GM),-22.734537,-2512.50,1.0
298,Visakhapatnam,Sankranthi,Personal Care,Atliq_Doodh_Kesar_Body_Lotion (200ML),39.290024,-9215.00,4.0


In [97]:
df.shape

(1500, 17)

In [98]:
top_products.to_csv('top10_items_based_on_category.csv')

In [99]:
df[ (df['city']=='Bengaluru') & (df['campaign_id']=='Diwali') ]  # .category.value_counts()

,event_id,store_id,campaign_id,product_code,base_price,promo_type,quantity_sold(before_promo),quantity_sold(after_promo),city,product_name,category,effective_price,quantity_change_percentage,revenue_before_promo,revenue_after_promo,incremental_revenue,ISU
1,a21f91,STBLR-8,Diwali,P03,156,25% OFF,393,322,Bengaluru,Atliq_Suflower_Oil (1L),Grocery & Staples,117.00,-18.066158,61308,37674.00,-23634.00,-71
4,1091cf,STBLR-6,Diwali,P05,55,25% OFF,108,93,Bengaluru,Atliq_Scrub_Sponge_For_Dishwash,Home Care,41.25,-13.888889,5940,3836.25,-2103.75,-15
5,8.02E+96,STBLR-4,Diwali,P11,190,50% OFF,91,116,Bengaluru,Atliq_Doodh_Kesar_Body_Lotion (200ML),Personal Care,95.00,27.472527,17290,11020.00,-6270.00,25
13,d290a1,STBLR-3,Diwali,P04,290,25% OFF,343,270,Bengaluru,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,217.50,-21.282799,99470,58725.00,-40745.00,-73
20,0f422c,STBLR-0,Diwali,P14,1020,BOGOF,42,168,Bengaluru,Atliq_waterproof_Immersion_Rod,Home Appliances,510.00,300.000000,42840,85680.00,42840.00,126
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,3b4a4f,STBLR-0,Diwali,P04,290,25% OFF,337,296,Bengaluru,Atliq_Farm_Chakki_Atta (1KG),Grocery & Staples,217.50,-12.166172,97730,64380.00,-33350.00,-41
1467,08086d,STBLR-7,Diwali,P10,65,50% OFF,120,188,Bengaluru,Atliq_Cream_Beauty_Bathing_Soap (125GM),Personal Care,32.50,56.666667,7800,6110.00,-1690.00,68
1474,e716e7,STBLR-7,Diwali,P09,110,50% OFF,78,117,Bengaluru,Atliq_Body_Milk_Nourishing_Lotion (120ML),Personal Care,55.00,50.000000,8580,6435.00,-2145.00,39
1483,b9b7dd,STBLR-3,Diwali,P12,62,50% OFF,126,157,Bengaluru,Atliq_Lime_Cool_Bathing_Bar (125GM),Personal Care,31.00,24.603175,7812,4867.00,-2945.00,31


In [100]:
import pandas as pd

# Assuming df is your DataFrame

# Convert 'effective_price' and other relevant columns to numeric
df['effective_price'] = pd.to_numeric(df['effective_price'])
df['quantity_change_percentage'] = pd.to_numeric(df['quantity_change_percentage'])
df['incremental_revenue'] = pd.to_numeric(df['incremental_revenue'])

# Group by city, campaign, category, and product name and sum the sales metrics
grouped_df = df.groupby(['city', 'campaign_id', 'category', 'product_name']).agg({
    'quantity_change_percentage': 'mean',
    'incremental_revenue': 'sum'
}).reset_index()

# Rank products within each category based on incremental revenue
grouped_df['rank'] = grouped_df.groupby(['city', 'campaign_id', 'category'])['incremental_revenue'].rank(ascending=False, method='max')

# Filter for the top 10 products in each category
grouped_df # [grouped_df['rank'] <= 10]

,city,campaign_id,category,product_name,quantity_change_percentage,incremental_revenue,rank
0,Bengaluru,Diwali,Combo1,Atliq_Home_Essential_8_Product_Combo,208.270351,18710000.00,1.0
1,Bengaluru,Diwali,Grocery & Staples,Atliq_Farm_Chakki_Atta (1KG),-12.423088,-324292.50,4.0
2,Bengaluru,Diwali,Grocery & Staples,Atliq_Masoor_Dal (1KG),51.184171,6358.84,2.0
3,Bengaluru,Diwali,Grocery & Staples,Atliq_Sonamasuri_Rice (10KG),52.738691,69488.00,1.0
4,Bengaluru,Diwali,Grocery & Staples,Atliq_Suflower_Oil (1L),-11.227077,-191997.00,3.0
...,...,...,...,...,...,...,...
295,Visakhapatnam,Sankranthi,Home Care,Atliq_Scrub_Sponge_For_Dishwash,-23.473684,-2090.00,3.0
296,Visakhapatnam,Sankranthi,Personal Care,Atliq_Body_Milk_Nourishing_Lotion (120ML),-22.450945,-7425.00,3.0
297,Visakhapatnam,Sankranthi,Personal Care,Atliq_Cream_Beauty_Bathing_Soap (125GM),-22.734537,-2512.50,1.0
298,Visakhapatnam,Sankranthi,Personal Care,Atliq_Doodh_Kesar_Body_Lotion (200ML),39.290024,-9215.00,4.0


In [102]:
df[ (df['city']=='Bengaluru') & (df['campaign_id']=='Diwali') & (df['category']=='Home Care')]['product_name'].nunique()

4